In [ ]:
# Test 1

# Download dataset


In [2]:
import os
import requests
from tqdm import tqdm

download_url = "https://zenodo.org/record/6473001/files/ArtDL.zip"
local_file_name = "ArtDL.zip"

# Check if the file already exists
if os.path.exists(local_file_name):
  print(f"The file '{local_file_name}' already exists. Skipping download.")

else:
  # Start the download
  print(f"Downloading the dataset from {download_url}...")
  
  head_response = requests.head(download_url)
  file_size = int(head_response.headers.get("content-length", 0))
  
  response = requests.get(download_url, stream=True)
  response.raise_for_status()
  
  # Initialize the progress bar
  with tqdm(total=file_size, unit="B", unit_scale=True, desc=local_file_name) as pbar:
    with open(local_file_name, "wb") as file:
      for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)
        pbar.update(len(chunk))  # Update progress bar
  
  print(f"Dataset downloaded and saved as '{local_file_name}'")

The file 'ArtDL.zip' already exists. Skipping download.


In [5]:
import zipfile
import os

# Name of the ZIP file
zip_file = "ArtDL.zip"

# Directory to extract the contents
extract_dir = "dataset"  # You can modify this directory name as needed

# Check if the file exists
if not os.path.exists(zip_file):
    print(f"The file '{zip_file}' does not exist. Please download it first.")
else:
    # Extract the ZIP file
    print(f"Extracting '{zip_file}' to '{extract_dir}'...")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extraction complete. Files are saved in '{extract_dir}'")

Extracting 'ArtDL.zip' to 'dataset'...
Extraction complete. Files are saved in 'dataset'


In [6]:
import os

# Directory containing the .jpg files
image_dir = "dataset/ArtDL/JPEGImages"

# Check if the directory exists
if not os.path.exists(image_dir):
    print(f"The directory '{image_dir}' does not exist.")
else:
    # List all files in the directory
    all_files = os.listdir(image_dir)
    
    # Filter only .jpg files (case-insensitive)
    jpg_files = [file for file in all_files if file.lower().endswith('.jpg')]
    
    # Print the count of .jpg files
    print(f"Number of .jpg files in '{image_dir}': {len(jpg_files)}")

Number of .jpg files in 'dataset/ArtDL/JPEGImages': 42479


In [7]:
import os
from PIL import Image

# Directory containing the .jpg files
image_dir = "dataset/ArtDL/JPEGImages"

# Array to store images
images = []

# Check if the directory exists
if not os.path.exists(image_dir):
    print(f"The directory '{image_dir}' does not exist.")
else:
    # List all files in the directory
    all_files = os.listdir(image_dir)
    
    # Filter only .jpg files (case-insensitive)
    jpg_files = [file for file in all_files if file.lower().endswith('.jpg')]
    
    # Load each image using Pillow and append to the array
    for file in jpg_files:
        file_path = os.path.join(image_dir, file)
        try:
            img = Image.open(file_path).convert("RGB")  # Ensure images are in RGB format
            images.append(img)
        except Exception as e:
            print(f"Error loading image {file_path}: {e}")

    print(f"Loaded {len(images)} images into the array.")

Loaded 42479 images into the array.


In [9]:
from transformers import CLIPModel, CLIPProcessor
from tqdm import tqdm

# Load the CLIP model and processor
model = CLIPModel.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K")
processor = CLIPProcessor.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K")

# Define the classes (supercategories)
classes = [
    "ANTONY OF PADUA",
    "JOHN THE BAPTIST",
    "PAUL",
    "FRANCIS",
    "MARY MAGDALENE",
    "JEROME",
    "DOMINIC",
    "MARY",
    "PETER",
    "SEBASTIAN"
]

# Preprocess the images and text
inputs = processor(
    text=classes,
    images=[image for image in tqdm(images, desc="Preprocessing images")],
    return_tensors="pt",
    padding=True,
    do_convert_rgb=False  
)

# Get model outputs
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # Image-text similarity scores
probs = logits_per_image.softmax(dim=1)  # Get label probabilities

print(probs)

Preprocessing images: 100%|██████████| 42479/42479 [00:00<00:00, 8533446.99it/s]


: 